In [ ]:
from os import path
import pandas as pd
import numpy as np
import importlib

# setting path
import os, sys
current_dir = os.path.abspath('')
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from source.utils.probing_tasks_utils import *
from source.load_data.wdc.load_wdc_dataset import EnglishDatasetLoader
from source.utils.levenshtein_distance import compute_lev_dist_for_pairs
from source.emb_extr_res.emb_extr_res import get_embeddings_df
from source.utils.visualization import plot_histogram
from source.probing.similarity_pairs_probing.get_data import get_pair_similarity_probing_task_df
from config import TYPE, SIZE, EMBEDDING_PATH, DATA_PATH, MODEL_TYPE
from source.emb_extr_res.emb_extr_res import get_embeddings_df, get_pairs_similarity_df, get_pretrain_agg_similarity
from source.load_data.wdc.load_wdc_dataset import EnglishDatasetLoader
from source.probing.brand_names import prepare_brands_list, brands_in_title_check, prepare_new_dataset, drop_brands
from source.probing.length import prepare_probing_len
from source.probing.words import words_in_title_check
from source.probing.load_data import load_files_probing_tasks

In [ ]:
print("TYPE:", TYPE)
print("SIZE:", SIZE)
print("MODEL:", MODEL_TYPE)

In [ ]:
# load all needed data

test_embeddings_path, train_embeddings_path, embedding_train_df, train_df = load_files_probing_tasks(EMBEDDING_PATH, DATA_PATH, TYPE, SIZE)

# Probing Task: Brands in Title

In [ ]:
# prepare brands name list 

brands_to_drop_cameras = [',', 'd','memory',  'photo', 'co', 'usa',  'power',  'digital', 'camera', 'cam',  'hd',  'a',  'inc',  'le',  'film',  'case',  'pro', 'cameras']
brands_to_drop_computers = ['transcend','null ,  seagate technology',  'null ,  lenovo', '150', 'mfr num cssd f120gblsb',  'corsair ,  null',  'null ,  apple', 'icy box', 'a data',  '216', 'mfr num dtse9g2 128gb',  '"TP-Link"@en-US',  '219',  'intel ,  null',  '1', 'n1', 'mfr num wds240g1g0b', '268', 'mfr num dtduo3c 32gb' , 'yli', 'g skill", ', 'null ,  amazon', 'mfr num mz 75e1t0b eu', 'null ,  crucial technology', 'null ,  samsung',  'null ,  hp', 'mfr num mtfddak512tbn 1ar1zabyy']

if TYPE == "cameras":
    brands_to_drop = brands_to_drop_cameras
elif TYPE == "computers":
    brands_to_drop = brands_to_drop_computers

# list with brand names
brands = prepare_brands_list(train_df, brands_to_drop)

### Prepare new dataset

In [ ]:
# # prepare dataset with randomly removed brand names from titles
# # dataset needs embeddings calulations - > Google Cloud

# new_dataset_brands, ids_of_removed_brands_offers = prepare_new_dataset(train_df, brands)

# new_dataset_brands_origin = new_dataset_brands[new_dataset_brands["changed"] == True].drop("changed", axis=1)

# # save data to use after embeddings extraction

# new_dataset_brands_origin.to_csv(os.path.join(DATA_PATH, f'dataset_brands_{TYPE}_{MODEL_TYPE}.csv'), index=False)

# np.save(os.path.join(DATA_PATH, 'ids_of_removed_brands_offers.npy'),ids_of_removed_brands_offers)

### Prepare dataframe for probing task

In [ ]:
# load data to use after embeddings extraction

new_dataset_brands_origin = pd.read_csv(os.path.join(DATA_PATH, f'dataset_brands_{TYPE}_{MODEL_TYPE}.csv'))
ids_of_removed_brands_offers = np.load(os.path.join(DATA_PATH, 'ids_of_removed_brands_offers.npy'))

# check brands in title

brands_in_title_df = brands_in_title_check(new_dataset_brands_origin, brands)
deleted_ids = brands_in_title_df[brands_in_title_df["brand_in_title"]==True]["id"].values

# load new embeddings - after brands removal

new_embeddings_brands = get_embeddings_df(os.path.join(EMBEDDING_PATH, 'new_embeddings_brands.csv'))


# prepare new dataframe with labels to train probing tasks

emb_brands = embedding_train_df[~embedding_train_df["id"].isin(ids_of_removed_brands_offers)] 

emb_removed_brands = new_embeddings_brands[new_embeddings_brands["id"].isin(ids_of_removed_brands_offers)]
df_probing_brands = pd.concat([emb_removed_brands, emb_brands])

df_probing_brands["label"] = df_probing_brands["id"].isin(deleted_ids)
df_probing_brands = df_probing_brands.drop(["id"], axis = 1)

In [ ]:
# save dataframe to train probing task - brands

df_probing_brands.to_csv(os.path.join(EMBEDDING_PATH, 'probing', 'df_probing_brands.csv'), index = False)